In [ ]:
%matplotlib inline

# Function chaos expansion.

Given a training dataset
whose input samples are generated from OpenTURNS probability distributions,
the [FCERegressor][gemseo.mlearning.regression.algos.fce.FCERegressor] can use any linear model fitting algorithm,
including sparse techniques,
to fit a functional chaos expansion (FCE) model of the form

$$Y = \sum_{i\in\mathcal{I}\subset\mathbb{N}^d} w_i\Psi_i(X)$$

where $\Psi_i(X)=\prod_{j=1}^d\psi_{i,j}(X_j)$
and $\mathbb{E}[\Psi_i(X)\Psi_j(X)]=\delta_{ij}$
with $\delta$ the Kronecker delta and $X$ a random vector.

A particular version of FCE is the polynomial chaos expansion (PCE)
for which the class [PCERegressor][gemseo.mlearning.regression.algos.pce.PCERegressor] interfaces
the OpenTURNS algorithm `openturns.FunctionalChaosAlgorithm`
(see the [OpenTURNS documentation](https://openturns.github.io/openturns/latest/user_manual/_generated/openturns.FunctionalChaosAlgorithm.html?highlight=functionalchaosalgorithm#openturns.FunctionalChaosAlgorithm)).

Note that FCE can also learn Jacobian data
in the hope of improving the quality of the surrogate model
for the same evaluation budget.

In this example,
we will compare different types of [FCERegressor][gemseo.mlearning.regression.algos.fce.FCERegressor]
to approximate the Ishigami function

$$f(X) = \sin(X_1) + 7\sin(X_2)^2 + 0.1X_3^4\sin(X_1)$$

where $X_1$, $X_2$ and $X_3$
are independent and uniformly distributed over the interval $[-\pi,\pi]$.


In [ ]:
from __future__ import annotations

from numpy import array

from gemseo import sample_disciplines
from gemseo.algos.doe.openturns.settings.ot_opt_lhs import OT_OPT_LHS_Settings
from gemseo.algos.doe.scipy.settings.mc import MC_Settings
from gemseo.datasets.dataset import Dataset
from gemseo.mlearning.linear_model_fitting.elastic_net_cv_settings import (
    ElasticNetCV_Settings,
)
from gemseo.mlearning.linear_model_fitting.lars_cv_settings import LARSCV_Settings
from gemseo.mlearning.linear_model_fitting.lasso_cv_settings import LassoCV_Settings
from gemseo.mlearning.linear_model_fitting.linear_regression_settings import (
    LinearRegression_Settings,
)
from gemseo.mlearning.linear_model_fitting.null_space_settings import NullSpace_Settings
from gemseo.mlearning.linear_model_fitting.omp_cv_settings import (
    OrthogonalMatchingPursuitCV_Settings,
)
from gemseo.mlearning.linear_model_fitting.ridge_cv_settings import RidgeCV_Settings
from gemseo.mlearning.linear_model_fitting.spgl1_settings import SPGL1_Settings
from gemseo.mlearning.regression.algos.fce import FCERegressor
from gemseo.mlearning.regression.algos.fce_settings import FCERegressor_Settings
from gemseo.mlearning.regression.algos.fce_settings import OrthonormalFunctionBasis
from gemseo.mlearning.regression.algos.pce import PCERegressor
from gemseo.mlearning.regression.algos.pce_settings import PCERegressor_Settings
from gemseo.mlearning.regression.quality.r2_measure import R2Measure
from gemseo.post.dataset.bars import BarPlot
from gemseo.problems.uncertainty.ishigami.ishigami_discipline import IshigamiDiscipline
from gemseo.problems.uncertainty.ishigami.ishigami_space import IshigamiSpace

First,
we define the Ishigami discipline and its uncertain space:



In [ ]:
discipline = IshigamiDiscipline()
uncertain_space = IshigamiSpace(IshigamiSpace.UniformDistribution.OPENTURNS)

and create a training dataset using an optimized latin hypercube sampling:



In [ ]:
training_dataset = sample_disciplines(
    [discipline],
    uncertain_space,
    "y",
    algo_settings_model=OT_OPT_LHS_Settings(n_samples=70, eval_jac=True),
)

as well as a validation dataset using Monte Carlo sampling:



In [ ]:
validation_dataset = sample_disciplines(
    [discipline],
    uncertain_space,
    "y",
    algo_settings_model=MC_Settings(n_samples=1000),
)

Then,
we create standard and gradient-enhanced FCEs
using an orthonormal polynomial basis (default basis)
with a maximum total degree of 7
and different regression techniques from scikit-learn to estimate the coefficients,
namely [ordinary least squares](https://scikit-learn.org/stable/modules/linear_model.html#ordinary-least-squares),
[ridge](https://scikit-learn.org/stable/modules/linear_model.html#regression) (i.e., L2 regularisation),
[lasso](https://scikit-learn.org/stable/modules/linear_model.html#lasso) (i.e., L1 regularisation),
[elasticnet](https://scikit-learn.org/stable/modules/linear_model.html#elastic-net) (i.e., L1 and L2 regularisation),
[least angle regression](https://scikit-learn.org/stable/modules/linear_model.html#least-angle-regression) (LARS)
and [orthogonal matching pursuit](https://scikit-learn.org/stable/modules/linear_model.html#orthogonal-matching-pursuit-omp).
Note that all these algorithms have been finely tuned using cross-validation,
except ordinary least squares regression for which there is no parameter to tune.
We also add the [SPGL1 algorithm](https://friedlander.io/spgl1)
to solve a basis pursuit denoise (BPN) problem,
as well as a null space algorithm.

!!! quote "References"
      Tiziano Ghisu, Diego I. Lopez, Pranay Seshadri and Shahrokh Shahpar.
      [Gradient-enhanced Least-square Polynomial Chaos Expansions for Uncertainty Quantification and Robust Optimization](https://arc.aiaa.org/doi/abs/10.2514/6.2021-3073).
      AIAA AVIATION FORUM, 2021.



In [ ]:
r2_learning = []
r2_validation = []
r2_learning_ge = []
r2_validation_ge = []
null_space_settings = NullSpace_Settings()
for linear_model_fitter_settings in [
    LinearRegression_Settings(),
    RidgeCV_Settings(),
    LassoCV_Settings(),
    ElasticNetCV_Settings(),
    LARSCV_Settings(),
    OrthogonalMatchingPursuitCV_Settings(),
    SPGL1_Settings(sigma=1e-7),
    null_space_settings,
]:
    if linear_model_fitter_settings == null_space_settings:
        # The null space technique requires gradient observations.
        r2_learning.append(0.0)
        r2_validation.append(0.0)
    else:
        # Train an FCE.
        fce_settings = FCERegressor_Settings(
            degree=7,
            linear_model_fitter_settings=linear_model_fitter_settings,
        )
        fce = FCERegressor(training_dataset, fce_settings)
        fce.learn()

        # Assess the quality of the FCE.
        r2 = R2Measure(fce)
        r2_learning.append(r2.compute_learning_measure().round(2)[0])
        r2_validation.append(r2.compute_test_measure(validation_dataset).round(2)[0])

    # Train a gradient-enhanced FCE.
    fce_settings = FCERegressor_Settings(
        degree=7,
        linear_model_fitter_settings=linear_model_fitter_settings,
        learn_jacobian_data=True,
    )
    fce = FCERegressor(training_dataset, fce_settings)
    fce.learn()

    # Assess the quality of the gradient-enhanced FCE.
    r2 = R2Measure(fce)
    r2_learning_ge.append(r2.compute_learning_measure().round(2)[0])
    r2_validation_ge.append(r2.compute_test_measure(validation_dataset).round(2)[0])

We create also a [PCERegressor][gemseo.mlearning.regression.algos.pce.PCERegressor]
using the LARS algorithm implemented in OpenTURNS:



In [ ]:
pce = PCERegressor(training_dataset, PCERegressor_Settings(degree=7, use_lars=True))
pce.learn()
r2 = R2Measure(pce)
r2_learning.append(r2.compute_learning_measure().round(2)[0])
r2_validation.append(r2.compute_test_measure(validation_dataset).round(2)[0])
r2_learning_ge.append(0)
r2_validation_ge.append(0)

From these results,
we can plot the quality of the different surrogate models,
expressed in terms of coefficient of determination $R^2$
(the higher, the better):



In [ ]:
dataset = Dataset()
dataset.add_group(
    "R2",
    array([r2_learning, r2_validation, r2_learning_ge, r2_validation_ge]),
    ("OLS", "L2", "L1", "L1-L2", "LARS", "OMP", "SPGL1", "NullSpace", "OT-LARS"),
)
dataset.index = ["Learning", "Validation", "Learning-GE", "Validation-GE"]

barplot = BarPlot(dataset, annotate=False)
barplot.execute(save=False)

First,
let us focus on the standard FCEs
that have not learned derivatives ("Learning" and "Validation" in the legend).
We can see that
the quality of learning is perfect, regardless of the method.
That's good, but not enough.
But what interests us is the quality of prediction of the validation dataset
to see if the surrogate model avoids overfitting.
In this regard,
ordinary least squares regression and ridge regression are wrong
while the other techniques are very good,
without really being able to tell them apart.
Now,
if we have a look to the gradient-enhanced FCEs
("Learning-GE" and "Validation-GE" in the legend).
we can see that the quality is significantly better, except for the LARS method.

Lastly,
these numerical experiments can be repeated
by replacing the polynomial basis with the Fourier series.



In [ ]:
r2_learning = []
r2_validation = []
r2_learning_ge = []
r2_validation_ge = []
null_space_settings = NullSpace_Settings()
for linear_model_fitter_settings in [
    LinearRegression_Settings(),
    RidgeCV_Settings(),
    LassoCV_Settings(),
    ElasticNetCV_Settings(),
    LARSCV_Settings(),
    OrthogonalMatchingPursuitCV_Settings(),
    SPGL1_Settings(sigma=1e-7),
    null_space_settings,
]:
    if linear_model_fitter_settings == null_space_settings:
        # The null space technique requires gradient observations.
        r2_learning.append(0.0)
        r2_validation.append(0.0)
    else:
        # Train an FCE.
        fce_settings = FCERegressor_Settings(
            degree=7,
            linear_model_fitter_settings=linear_model_fitter_settings,
            basis=OrthonormalFunctionBasis.FOURIER,
        )
        fce = FCERegressor(training_dataset, fce_settings)
        fce.learn()

        # Assess the quality of the FCE.
        r2 = R2Measure(fce)
        r2_learning.append(r2.compute_learning_measure().round(2)[0])
        r2_validation.append(r2.compute_test_measure(validation_dataset).round(2)[0])

    # Train a gradient-enhanced FCE.
    fce_settings = FCERegressor_Settings(
        degree=7,
        linear_model_fitter_settings=linear_model_fitter_settings,
        basis=OrthonormalFunctionBasis.FOURIER,
        learn_jacobian_data=True,
    )
    fce = FCERegressor(training_dataset, fce_settings)
    fce.learn()

    # Assess the quality of the gradient-enhanced FCE.
    r2 = R2Measure(fce)
    r2_learning_ge.append(r2.compute_learning_measure().round(2)[0])
    r2_validation_ge.append(r2.compute_test_measure(validation_dataset).round(2)[0])

dataset = Dataset()
dataset.add_group(
    "R2",
    array([r2_learning, r2_validation, r2_learning_ge, r2_validation_ge]),
    ("OLS", "L2", "L1", "L1-L2", "LARS", "OMP", "SPGL1", "NullSpace"),
)
dataset.index = ["Learning", "Validation", "Learning-GE", "Validation-GE"]

barplot = BarPlot(dataset, annotate=False)
barplot.execute(save=False)

We then see the same type of ranking, with even better validation qualities.
This can be easily explained by the nature of Ishigami's function,
in which trigonometric terms are important.
Furthermore,
learning Jacobian significantly improves the quality of surrogate models
in the case of ridge regression and ordinary least squares.

